# Análisis de Resultados Electorales 2025

Este notebook contiene el análisis de los datos de las elecciones legislativas 2025.

In [ ]:
# Instalar las librerías necesarias (ejecutar solo una vez)
%pip install duckdb pandas

In [1]:
# Importar las librerías necesarias
import duckdb
import pandas as pd

# Leer el archivo parquet usando DuckDB
archivo = 'elecciones_legislativas_2025/ResultadosElectorales.parquet'

# Crear una conexión a DuckDB
conn = duckdb.connect()

# Leer el archivo parquet
df = conn.execute(f"SELECT * FROM '{archivo}'").df()

# Mostrar información básica del dataset
print("=" * 80)
print("INFORMACIÓN DEL DATASET")
print("=" * 80)
print(f"\nDimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
print(f"\nColumnas: {list(df.columns)}")
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\n{'=' * 80}")
print("PRIMERAS 10 FILAS")
print("=" * 80)
df.head(10)

INFORMACIÓN DEL DATASET

Dimensiones: 2295376 filas x 26 columnas

Columnas: ['año', 'eleccion_tipo', 'recuento_tipo', 'padron_tipo', 'distrito_id', 'distrito_nombre', 'seccionprovincial_id', 'seccionprovincial_nombre', 'seccion_id', 'seccion_nombre', 'circuito_id', 'circuito_nombre', 'mesa_id', 'mesa_tipo', 'mesa_electores', 'cargo_id', 'cargo_nombre', 'agrupacion_id', 'agrupacion_nombre', 'lista_numero', 'lista_nombre', 'votos_tipo', 'votos_cantidad', 'estado_final', 'eleccion_id', 'recuento_id']

Tipos de datos:
año                         object
eleccion_tipo               object
recuento_tipo               object
padron_tipo                 object
distrito_id                 object
distrito_nombre             object
seccionprovincial_id        object
seccionprovincial_nombre    object
seccion_id                  object
seccion_nombre              object
circuito_id                 object
circuito_nombre             object
mesa_id                     object
mesa_tipo               

,año,eleccion_tipo,recuento_tipo,padron_tipo,distrito_id,distrito_nombre,seccionprovincial_id,seccionprovincial_nombre,seccion_id,seccion_nombre,...,cargo_nombre,agrupacion_id,agrupacion_nombre,lista_numero,lista_nombre,votos_tipo,votos_cantidad,estado_final,eleccion_id,recuento_id
0,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,120,FUERZA PATRIA,None,None,POSITIVO,89,ESCRUTADO,2,1
1,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,6,ALIANZA LA LIBERTAD AVANZA,None,None,POSITIVO,74,ESCRUTADO,2,1
2,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,31,FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD,None,None,POSITIVO,17,ESCRUTADO,2,1
3,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,268,CIUDADANOS UNIDOS,None,None,POSITIVO,9,ESCRUTADO,2,1
4,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,7,PARTIDO FEDERAL,None,None,POSITIVO,3,ESCRUTADO,2,1
5,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,4,ALIANZA POTENCIA,None,None,POSITIVO,1,ESCRUTADO,2,1
6,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,11,FRENTE PATRIOTA FEDERAL,None,None,POSITIVO,1,ESCRUTADO,2,1
7,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,2,GEN,None,None,POSITIVO,1,ESCRUTADO,2,1
8,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,267,HAGAMOS FUTURO,None,None,POSITIVO,1,ESCRUTADO,2,1
9,2025,GENERALES,PROVISORIO,NORMAL,1,CIUDAD AUTÓNOMA DE BUENOS AIRES,None,None,1,COMUNA 1,...,SENADOR NACIONAL,3,LA IZQUIERDA EN LA CIUDAD,None,None,POSITIVO,1,ESCRUTADO,2,1


In [3]:
# Obtener todas las secciones únicas de CABA
secciones_caba = df[df['distrito_nombre'] == 'CIUDAD AUTÓNOMA DE BUENOS AIRES']['seccion_nombre'].unique()

print("SECCIONES EN CIUDAD AUTÓNOMA DE BUENOS AIRES")
print("=" * 60)
for seccion in sorted(secciones_caba):
    print(seccion)

SECCIONES EN CIUDAD AUTÓNOMA DE BUENOS AIRES
COMUNA 1
COMUNA 10
COMUNA 11
COMUNA 12
COMUNA 13
COMUNA 14
COMUNA 15
COMUNA 2
COMUNA 3
COMUNA 4
COMUNA 5
COMUNA 6
COMUNA 7
COMUNA 8
COMUNA 9


In [2]:
# Convertir votos_cantidad a entero numérico
df['votos_cantidad'] = pd.to_numeric(df['votos_cantidad'], errors='coerce').fillna(0).astype(int)

# Agrupar por partido y sumar los votos
votos_por_partido = (
    df.groupby("agrupacion_nombre")["votos_cantidad"]
    .sum()
    .reset_index()
    .sort_values(by="votos_cantidad", ascending=False)
)

# Renombrar columnas
votos_por_partido.columns = ['Partido', 'Votos']

# Mostrar el resultado completo
print(votos_por_partido.to_string(index=False))


                                           Partido   Votos
                        ALIANZA LA LIBERTAD AVANZA 8830888
                             ALIANZA FUERZA PATRIA 3657285
                                     FUERZA PATRIA 2851410
                                LA LIBERTAD AVANZA 2704662
    FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD  997097
                         ALIANZA PROVINCIAS UNIDAS  762798
                        FRENTE CÍVICO POR SANTIAGO  606990
                            FRENTE TUCUMÁN PRIMERO  524057
                                 FUERZA ENTRE RÍOS  515702
                              PRIMERO LOS SALTEÑOS  395426
                                 PROVINCIAS UNIDAS  330089
                      FUERZA JUSTICIALISTA MENDOZA  253109
                  PROPUESTA FEDERAL PARA EL CAMBIO  243326
                                    LA NEUQUINIDAD  240697
                    FRENTE FUERZA PATRIA PERONISTA  227344
                       JUNTOS DEFENDEMOS RÍO NEGRO  2082